In [ ]:


# -- IMPORTS --
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import re
from collections import defaultdict
from tqdm import tqdm
import numpy as np
from scipy.stats import pearsonr
import openpyxl
from datetime import datetime
import yfinance as yf
import os
from plotly.subplots import make_subplots
pio.renderers.default = 'notebook' # viz outputs in the notebook

# -- M2 DATA IMPORT --
file_path = "data/bq_magento.csv"  
data_m2 = pd.read_csv(file_path)
data_m2['datetime'] = pd.to_datetime(data_m2['datetime'])
data_m2 = data_m2.drop(columns=['quantity'])


# -- GA4 DATA IMPORT --
file_path = "data/bq_ga4.csv"   
data_ga4 = pd.read_csv(file_path)
data_ga4['datetime'] = pd.to_datetime(data_ga4['datetime'])

# -- Build GA4 Source table --
data_ga4_source = data_ga4.groupby(
    ['transaction_id', 'datetime']
).agg(
    traffic_source=('traffic_source', 'first'),        
    traffic_medium=('traffic_medium', 'first'),        
    traffic_name=('traffic_name', 'first'),          
    revenue=('revenue', 'first'),       
    quantity=('quantity', 'first')  
).reset_index()
# Capitalize traffic source and replace (direct)
data_ga4_source['traffic_source'] = data_ga4_source['traffic_source'].str.capitalize()
data_ga4_source['traffic_source'] = data_ga4_source['traffic_source'].replace('(direct)', 'Direct')

# -- Build GA4 Item table --
data_ga4_item = data_ga4[['transaction_id', 'datetime', 'item_name', 
                          'category_name1', 'category_name2', 'category_name3', 'category_name4', 'category_name5', 
                          'item_revenue', 'item_quantity' ]].copy()
# Lowercase and title the category columns
for col in ["category_name1", "category_name2", "category_name3", "category_name4"]:
    data_ga4_item[col] = data_ga4_item[col].str.lower()
for col in ["category_name1", "category_name2", "category_name3", "category_name4"]:
    data_ga4_item[col] = data_ga4_item[col].str.title()


In [ ]:
# -- UNIFY USERS ie. create user_id --
def clean_name_address(text: str) -> str:
    """
    1) Lowercase
    2) Remove all NON-alphanumeric characters (anything except letters, digits, and spaces),
       replacing them with a single space
    3) Replace multiple spaces with a single space
    4) Strip leading/trailing spaces
    """
    if not isinstance(text, str):
        text = str(text) if pd.notnull(text) else ""
    
    # Lowercase
    text = text.lower().strip()
    
    # Replace any sequence of non-alphanumeric (including punctuation) with a space
    # Here [^a-z0-9 ] means "any char that is NOT (a to z or 0-9 or space)".
    text = re.sub(r'[^a-z0-9 ]+', ' ', text)
    
    # Collapse multiple spaces into one
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def clean_email(text: str) -> str:
    """
    1) Lowercase
    2) Strip leading/trailing spaces
    
    We do NOT remove non-alphanumeric chars from email because
    valid addresses may contain '.', '+', '_', etc. 
    """
    if not isinstance(text, str):
        text = str(text) if pd.notnull(text) else ""
    
    return text.lower().strip()


def generate_user_id(df):
    user_mapping = {}
    user_counter = 1
    user_ids = []
    
    df['user_name'] = df['user_name'].apply(clean_name_address)
    df['shipping_address'] = df['shipping_address'].apply(clean_name_address)
    df['email'] = df['email'].apply(clean_email)
        
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Users"):
 
        email, name, address = row['email'], row['user_name'], row['shipping_address']

        # Priority: Same email = same user
        if email in user_mapping:
            user_ids.append(user_mapping[email])
        
        # Otherwise, check for same name + same address
        elif (name, address) in user_mapping:
            user_ids.append(user_mapping[(name, address)])
        
        # Otherwise, create a new user_id
        else:
            user_mapping[email] = user_counter
            user_mapping[(name, address)] = user_counter
            user_ids.append(user_counter)
            user_counter += 1
    
    df['user_id'] = pd.Series(user_ids, dtype=str)
    return df

data_m2 = generate_user_id(data_m2)

# Merge data_m2 and data_ga4_item using a left join on the 'transaction_id' column
data_ga4_item = pd.merge(data_ga4_item, data_m2, on='transaction_id', how='left')


In [ ]:
# EXPLORATION

# -----------------------------------------------------------------------------------------
# -- Viz currencies --
data_m2.groupby(['currency']).agg(revenue=('revenue', 'sum')) \
    .assign(percent_revenue=lambda x: ((x['revenue'] / x['revenue'].sum()) * 100).round(1)) \
    .sort_values(by='revenue', ascending=False)

In [ ]:
# ------------------------- Choose the Currency for the analysis -------------------------------
currency_to_choose = 'EUR'
# -----------------------------------------------------------------------------------------

In [ ]:

# -----------------------------------------------------------------------------------------
# -- Viz top emails --

emails = data_m2.groupby('email')['transaction_id'].count()
emails = emails.sort_values(ascending=False)
emails.head(20)

In [ ]:
# ------------------------- Remove wrong or internal emails -------------------------------
email_keywords_to_ignore = ['qwertee', 'test']
# -----------------------------------------------------------------------------------------

pattern = '|'.join(email_keywords_to_ignore)
data_m2 = data_m2[~data_m2['user_id'].str.contains(pattern, case=False, na=False)]

In [ ]:
# -----------------------------------------------------------------------------------------
# -- Viz top categories --

# Total unique transactions
total_txn = data_ga4_item["transaction_id"].nunique()

# Helper function to get percentage of transactions per category
def category_percentages(df, col):
    counts = df.groupby(col)["transaction_id"].nunique().sort_values(ascending=False)
    pct = (counts / total_txn) * 100
    pct = pct.round(1)  # Round to 1 decimal
    pct = pct.astype(str) + "%"  # Add the '%' sign
    return pct.reset_index(name="percentage_of_transactions")

# For each category column, compute + sort by percentage
cat1_df = category_percentages(data_ga4_item, "category_name1")
cat2_df = category_percentages(data_ga4_item, "category_name2")
cat3_df = category_percentages(data_ga4_item, "category_name3")
cat4_df = category_percentages(data_ga4_item, "category_name4")

cat1_df.columns = ["category_name1", "pct1"]
cat2_df.columns = ["category_name2", "pct2"]
cat3_df.columns = ["category_name3", "pct3"]
cat4_df.columns = ["category_name4", "pct4"]

categ_df = pd.concat(
    [cat1_df.reset_index(drop=True),
     cat2_df.reset_index(drop=True),
     cat3_df.reset_index(drop=True),
     cat4_df.reset_index(drop=True)],
    axis=1
)

categ_df.head(5)

In [ ]:
# ------------------ Keep category level that makes sense ---------------------------------
category_to_analyse = "category_name1"
# -----------------------------------------------------------------------------------------

In [ ]:
# -----------------------------------------------------------------------------------------
# -- Viz top order status --

# Lowercase the order_status (optional)
data_m2["order_status"] = data_m2["order_status"].str.lower()
# Calculate total unique transactions
total_txn = data_m2["transaction_id"].nunique()
# Group by order_status:
grouped = data_m2.groupby("order_status").agg(
    total_revenue=("revenue", "sum"),
    unique_transactions=("transaction_id", "nunique")
).reset_index()
# Compute percentage of unique transactions
grouped["percentage_of_transactions"] = (
    grouped["unique_transactions"] / total_txn * 100
)
# Sort descending by percentage of transactions
order_status_df = grouped.sort_values("percentage_of_transactions", ascending=False)

order_status_df.head(10)

In [ ]:
# ------------ Remmove order status that show refunds or cancelations ---------------------
order_status_keyWords_Refund_Cancel = ["cancel", "refund"]
# -----------------------------------------------------------------------------------------

In [ ]:
# -----------------------------------------------------------------------------------------
# -- Viz date range --
print(f"Source: Magento 2 from {data_m2['datetime'].min().strftime('%b %d, %Y')} to {data_m2['datetime'].max().strftime('%b %d, %Y')}")
print(f"Source: GA4 from {data_ga4_source['datetime'].min().strftime('%b %d, %Y')} to {data_ga4_source['datetime'].max().strftime('%b %d, %Y')}")

In [ ]:
# -- Convert multiple currency revenue to one --
"""
Converts the revenue column in the dataframe to the specified base currency using historical exchange rates.
Rounds the converted revenue to 2 decimals.
"""
# Create date column for merging
data_m2['date'] = data_m2['datetime'].dt.date

# Get min and max dates from the data (with a small buffer)
start_date = data_m2['datetime'].min() - pd.Timedelta(days=7)
end_date = data_m2['datetime'].max() + pd.Timedelta(days=7)

print(f"Data date range: {data_m2['datetime'].min()} to {data_m2['datetime'].max()}")
print(f"Fetching exchange rates from {start_date} to {end_date}")

# Get unique currencies in the dataframe
unique_currencies = data_m2['currency'].unique()
print(f"Unique currencies found: {unique_currencies}")

# Dictionary to store exchange rate data for each currency pair
exchange_rates_dict = {}

# Fetch historical exchange rates for each currency to the base currency
for currency in unique_currencies:
    if currency == currency_to_choose:
        # If the currency is the base currency, set rate = 1
        exchange_rates_dict[currency] = pd.Series(1, index=pd.date_range(start=start_date, end=end_date, freq='D'))
 
    else:
        currency_pair = currency + currency_to_choose + '=X'
        print(f"Fetching exchange rates for: {currency_pair}")

        # Download historical exchange rates
        rates_df = yf.download(
            currency_pair, 
            start=start_date.strftime('%Y-%m-%d'), 
            end=end_date.strftime('%Y-%m-%d'), 
            interval='1d'
        )['Close']
        
        # Convert index to date for merging
        rates_df.index = rates_df.index.date
        exchange_rates_dict[currency] = rates_df

# Create a result dataframe
data_m2['revenue_converted'] = data_m2['revenue']  # Initialize with original values

# Ensure 'date' column in data_m2 is in datetime format
data_m2['date'] = pd.to_datetime(data_m2['date'])

# Loop through exchange rates and update the 'rate' column
for currency, rates in exchange_rates_dict.items():
    rates.index = pd.to_datetime(rates.index)
    rates = rates.reindex(pd.date_range(start=rates.index.min(), end=rates.index.max(), freq='D')).ffill()
    rate_mapping = rates.squeeze().to_dict()
    
    mask = data_m2['currency'] == currency
    data_m2.loc[mask, 'rate'] = data_m2.loc[mask, 'date'].map(rate_mapping)

# If the currency is the base currency, set the rate to 1
mask_base_currency = data_m2['currency'] == currency_to_choose
data_m2.loc[mask_base_currency, 'rate'] = 1

data_m2['revenue_converted'] = data_m2['revenue'] * data_m2['rate']

# Keep only usefull columns
data_m2 = data_m2[['transaction_id', 'user_id', 'datetime', 'revenue_converted', 'discount', 'order_status']]


In [ ]:
# -- 1.1 REVENUE TIMELINE CHART (Grouped by Month) --

# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()

# Ensure the datetime column is in the correct format
data_m2_noCancel_noRefund['datetime'] = pd.to_datetime(data_m2_noCancel_noRefund['datetime'])

# Group the data by month and sum the revenue
# Use the 'datetime' column and extract the month and year to group by
revenue_timeline = data_m2_noCancel_noRefund.groupby(data_m2_noCancel_noRefund['datetime'].dt.to_period('M')).agg({'revenue_converted': 'sum'}).reset_index()

# Convert period to a regular datetime for better plotting
revenue_timeline['datetime'] = revenue_timeline['datetime'].dt.to_timestamp()

# AFTER you've created revenue_timeline, remove the last row if it corresponds to the current (incomplete) month:
current_month = pd.Timestamp.now().month
current_year = pd.Timestamp.now().year

# Check if the last row in revenue_timeline is from the current month/year, and drop if so
if (revenue_timeline.iloc[-1]['datetime'].month == current_month and
    revenue_timeline.iloc[-1]['datetime'].year == current_year):
    revenue_timeline = revenue_timeline.iloc[:-1]
    
# Create the Plotly figure
fig_revenue_timeline = go.Figure()

# Add a trace for the revenue timeline
fig_revenue_timeline.add_trace(go.Scatter(
    x=revenue_timeline['datetime'],  
    y=revenue_timeline['revenue_converted'],  
    mode='lines',                 
    name='Revenue Over Time',
    line=dict(color='lightblue')  
))

# Customize the layout
fig_revenue_timeline.update_layout(
    title='Revenue Over Time (Grouped by Month)',
    xaxis_title='',
    yaxis_title='Total Revenue ($)',
    template='plotly_white',
    width=900,
    height=500
)

fig_revenue_timeline.show()


In [ ]:
# -- 1.2 REVENUE TIMELINE CHART (Grouped by Month) --

# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()

# Ensure the datetime column is in the correct format
data_m2_noCancel_noRefund.loc[:, 'datetime'] = pd.to_datetime(data_m2_noCancel_noRefund['datetime'])


# -- MONTH Data Preparation --
# Extract month and group by month, summing the revenue for each month
data_m2_noCancel_noRefund.loc[:, 'month'] = data_m2_noCancel_noRefund['datetime'].dt.month

# Group by month and sum the revenue
revenue_by_month = data_m2_noCancel_noRefund.groupby('month').agg({'revenue_converted': 'sum'}).reset_index()

# Calculate the total revenue
total_revenue = revenue_by_month['revenue_converted'].sum()

# Calculate the revenue percentage for each month
revenue_by_month['revenue_percentage'] = (revenue_by_month['revenue_converted'] / total_revenue) * 100

# Handle the monthly average revenue
monthly_average = revenue_by_month['revenue_percentage'].mean()

# Map month number to month name
month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
revenue_by_month['month_name'] = revenue_by_month['month'].apply(lambda x: month_names[x-1])



# -- WEEK Data Preparation --
# Extract the weekday (0 = Monday, 1 = Tuesday, ..., 6 = Sunday)
data_m2_noCancel_noRefund.loc[:, 'weekday'] = data_m2_noCancel_noRefund['datetime'].dt.weekday

# Group by weekday and sum the revenue for each weekday (now including Saturday and Sunday)
revenue_by_weekday = data_m2_noCancel_noRefund.groupby('weekday').agg({'revenue_converted': 'sum'}).reset_index()

# Calculate the total revenue
total_revenue = revenue_by_weekday['revenue_converted'].sum()

# Calculate the revenue percentage for each weekday
revenue_by_weekday['revenue_percentage'] = (revenue_by_weekday['revenue_converted'] / total_revenue) * 100

# Handle the weekday average revenue percentage
weekday_average = revenue_by_weekday['revenue_percentage'].mean()

# Map weekday number to weekday name (including Saturday and Sunday)
weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
revenue_by_weekday['weekday_name'] = revenue_by_weekday['weekday'].apply(lambda x: weekday_names[x])





# Create subplots with shared y-axis
fig_revenue_distributin_month_and_weekday = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Monthly Revenue Percentage', 'Weekday Revenue Percentage'),
    shared_yaxes=True
)

# Add monthly revenue bars to the left subplot
fig_revenue_distributin_month_and_weekday.add_trace(
    go.Bar(
        x=revenue_by_month['month_name'],
        y=revenue_by_month['revenue_percentage'],
        name='Monthly Revenue',
        marker=dict(color='lightblue'),
        showlegend=False  # Hide legend
    ),
    row=1, col=1
)

# Add monthly average line to the left subplot
fig_revenue_distributin_month_and_weekday.add_trace(
    go.Scatter(
        x=revenue_by_month['month_name'],
        y=[monthly_average] * len(revenue_by_month),
        mode='lines',
        name='Monthly Average',
        line=dict(color='blue', dash='dash'),
        showlegend=False  # Hide legend
    ),
    row=1, col=1
)

# Add weekday revenue bars to the right subplot
fig_revenue_distributin_month_and_weekday.add_trace(
    go.Bar(
        x=revenue_by_weekday['weekday_name'],
        y=revenue_by_weekday['revenue_percentage'],
        name='Weekday Revenue',
        marker=dict(color='#75B7E5'),
        showlegend=False  # Hide legend
    ),
    row=1, col=2
)

# Add weekday average line to the right subplot
fig_revenue_distributin_month_and_weekday.add_trace(
    go.Scatter(
        x=revenue_by_weekday['weekday_name'],
        y=[weekday_average] * len(revenue_by_weekday),
        mode='lines',
        name='Weekday Average',
        line=dict(color='blue', dash='dash'),
        showlegend=False  # Hide legend
    ),
    row=1, col=2
)

# Update layout
fig_revenue_distributin_month_and_weekday.update_layout(
    showlegend=False,  # Hide legend
    template='plotly_white',
    width=1200,
    height=500,
    yaxis_title='Revenue Percentage (%)',
    margin=dict(t=30)  # Reduce top margin since we removed the title
)

# Update x-axis properties for both subplots
fig_revenue_distributin_month_and_weekday.update_xaxes(tickangle=45, row=1, col=1)
fig_revenue_distributin_month_and_weekday.update_xaxes(tickangle=45, row=1, col=2)

# Show the figure
fig_revenue_distributin_month_and_weekday.show()

In [ ]:
# -- 2. CLIENT REPARTITION CHART --

# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()

# Group by customer and sum the revenue (Grand Total)
client_revenue = data_m2_noCancel_noRefund.groupby('user_id')['revenue_converted'].sum()

# Sort by revenue in descending order
client_revenue = client_revenue.sort_values(ascending=False)

# Calculate the percentage of revenue per client
client_revenue_percentage = client_revenue / client_revenue.sum()

# Limit the data to top 1000 clients
client_revenue_percentage = client_revenue_percentage[:1000]

# Create the Plotly figure
fig_client_revenue = go.Figure()

# Use a bar chart instead of a line chart
fig_client_revenue.add_trace(go.Bar(
    x=list(range(1, len(client_revenue_percentage) + 1)),  # x-axis: clients ordered by revenue
    y=client_revenue_percentage * 100,  # y-axis: the percentage of revenue by each client
    name='% of Revenue by Client',
    marker=dict(color='blue')  # Set color to blue
))

# Customize the layout
fig_client_revenue.update_layout(
    title='Revenue by Client',
    xaxis_title='Clients Ordered by Revenue',
    yaxis_title='% of Revenue (Lifetime Value)',
    yaxis=dict(tickformat='.0f%'),
    template='plotly_white',
    width=900,
    height=500,
    bargap=0
)

fig_client_revenue.show()


In [ ]:
# -- 3. CLIENT REPARTITION --

# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()
# Group and sort revenues by customer
client_revenue = data_m2_noCancel_noRefund.groupby('user_id')['revenue_converted'].sum()
client_revenue_sorted = client_revenue.sort_values(ascending=False)

# Calculate total
total_revenue = client_revenue_sorted.sum()
total_clients = len(client_revenue_sorted)


# Calculate the number of clients in each group
top_100_clients_count = client_revenue_sorted.head(100).count()
top_1_percent_count = int(len(client_revenue_sorted) * 0.01)
top_10_percent_count = int(len(client_revenue_sorted) * 0.10)
top_25_percent_count = int(len(client_revenue_sorted) * 0.25)
top_50_percent_count = int(len(client_revenue_sorted) * 0.50)
top_75_percent_count = int(len(client_revenue_sorted) * 0.75)



# Calculate revenue for each group
top_100_clients_revenue = client_revenue_sorted.head(100).sum()
top_1_percent_revenue = client_revenue_sorted.head(top_1_percent_count).sum()
top_10_percent_revenue = client_revenue_sorted.head(top_10_percent_count).sum()
top_25_percent_revenue = client_revenue_sorted.head(top_25_percent_count).sum()
top_50_percent_revenue = client_revenue_sorted.head(top_50_percent_count).sum()
top_75_percent_revenue = client_revenue_sorted.head(top_75_percent_count).sum()

# Create the table
client_repartition_df = pd.DataFrame({
    "Clients": ["Top 100 Clients", "Top 1%", "Top 10%", "Top 25%", "Top 50%", "Top 75%", "All Clients"],
        "Number of Clients": [
        top_100_clients_count, 
        top_1_percent_count, 
        top_10_percent_count, 
        top_25_percent_count, 
        top_50_percent_count, 
        top_75_percent_count, 
        total_clients
    ],
    "% of Revenue": [
        f"{round((top_100_clients_revenue / total_revenue) * 100, 1)}%", 
        f"{round((top_1_percent_revenue / total_revenue) * 100, 1)}%", 
        f"{round((top_10_percent_revenue / total_revenue) * 100, 1)}%", 
        f"{round((top_25_percent_revenue / total_revenue) * 100, 1)}%", 
        f"{round((top_50_percent_revenue / total_revenue) * 100, 1)}%", 
        f"{round((top_75_percent_revenue / total_revenue) * 100, 1)}%", 
        "100%"
    ],
    "Revenue": [
        f"${top_100_clients_revenue:,.0f}", 
        f"${top_1_percent_revenue:,.0f}", 
        f"${top_10_percent_revenue:,.0f}", 
        f"${top_25_percent_revenue:,.0f}", 
        f"${top_50_percent_revenue:,.0f}", 
        f"${top_75_percent_revenue:,.0f}", 
        f"${total_revenue:,.0f}"
    ],

})

# Display the table
client_repartition_df

In [ ]:
# -- 4. TOP 100 CLIENTS REVENUE CHART --

# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()

# Group by customer and sum the revenue (Grand Total)
client_revenue = data_m2_noCancel_noRefund.groupby('user_id')['revenue_converted'].sum()

# Sort by revenue in descending order
client_revenue = client_revenue.sort_values(ascending=False)

# Limit the data to top 100 clients
top_100_revenue = client_revenue.head(100)

# Create the Plotly figure
fig_top_client_revenue = go.Figure()

# Use a bar chart instead of a line chart
fig_top_client_revenue.add_trace(go.Bar(
    x=list(range(1, len(top_100_revenue) + 1)),  
    y=top_100_revenue,  
    name='Revenue by Client',
    marker=dict(color='lightblue')  
))

# Customize the layout
fig_top_client_revenue.update_layout(
    title='Revenue for Top 100 Clients',
    xaxis_title='Clients (Ordered by Revenue)',
    yaxis_title='Revenue',
    yaxis=dict(tickprefix='$'),  
    template='plotly_white',
    width=900,
    height=500,
)

fig_top_client_revenue.show()

In [ ]:
# -- 5. TOP 100 CLIENTS TRANSACTIONS CHART --

# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()

# Group by customer and sum the revenue (Grand Total)
client_transaction = data_m2_noCancel_noRefund.groupby('user_id')['transaction_id'].count()

# Sort by revenue in descending order
client_transaction = client_transaction.sort_values(ascending=False)

# Limit the data to top 100 clients
top_100_transaction = client_transaction.head(100)

# Create the Plotly figure
fig_top_client_transactions = go.Figure()

# Use a bar chart instead of a line chart
fig_top_client_transactions.add_trace(go.Bar(
    x=list(range(1, len(top_100_transaction) + 1)), 
    y=top_100_transaction,  
    name='Number of Transactions by Client',
    marker=dict(color='lightgreen')  
))

# Customize the layout
fig_top_client_transactions.update_layout(
    title='Number of Transactions for Top 100 Clients',
    xaxis_title='Clients (Ordered by Number of Transactions)',
    yaxis_title='Total Number of Transactions',
    yaxis=dict(tickformat="d"),  # Ensure the y-axis shows integer values
    template='plotly_white',
    width=900,
    height=500,
)

fig_top_client_transactions.show()

In [ ]:
# -- 6. MAIN KPIs TOP 1% CLIENTS VS BOTTOM 99% --

# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False)
    & (data_m2['revenue_converted'] > 0)
].copy()

# Convert 'revenue_converted' => numeric
data_m2_noCancel_noRefund['Grand Total (Base)'] = pd.to_numeric(
    data_m2_noCancel_noRefund['revenue_converted'],
    errors='coerce'
)

# Sort for consistent "first" purchase identification
data_m2_noCancel_noRefund = data_m2_noCancel_noRefund.sort_values(by=['user_id', 'datetime'])

# BUILD CUSTOMER-LEVEL TABLE
client_data = data_m2_noCancel_noRefund.groupby('user_id').agg({
    'Grand Total (Base)': 'sum',
    'transaction_id': 'nunique',
    'revenue_converted': 'first'
})

# Rename columns for clarity
client_data.rename(columns={
    'transaction_id': '# of Orders',
    'revenue_converted': 'First Purchase Value'
}, inplace=True)

# Sort by total revenue descending
client_data = client_data.sort_values('Grand Total (Base)', ascending=False)

# Identify Top 1% and Bottom 99% clients
top_1_percent_count = int(len(client_data) * 0.01)
top_1_percent_clients = client_data.head(top_1_percent_count)
bottom_99_percent_clients = client_data.tail(len(client_data) - top_1_percent_count)
top_1_percent_user_ids = top_1_percent_clients.index
bottom_99_percent_user_ids = bottom_99_percent_clients.index

# SEPARATE DATA FOR METRICS
# Data with canceled/refunded orders
data_top_1_percent = data_m2[data_m2['user_id'].isin(top_1_percent_user_ids)].copy()
data_bottom_99_percent = data_m2[data_m2['user_id'].isin(bottom_99_percent_user_ids)].copy()

# Data without canceled/refunded orders
data_top_1_percent_clean = data_m2_noCancel_noRefund[
    data_m2_noCancel_noRefund['user_id'].isin(top_1_percent_user_ids)
].copy()
data_bottom_99_percent_clean = data_m2_noCancel_noRefund[
    data_m2_noCancel_noRefund['user_id'].isin(bottom_99_percent_user_ids)
].copy()

def safe_div(numerator, denominator):
    """Avoid ZeroDivisionError; return 0 if denominator=0."""
    return numerator / denominator if denominator else 0

# Calculate Cancellation and Refund Rates
def calculate_rates(data):
    cancellations = data[data['order_status'].str.contains('cancel', case=False, na=False)]
    refunds = data[data['order_status'].str.contains('refund', case=False, na=False)]
    return {
        'cancellation_rate': 100 * safe_div(len(cancellations), len(data)),
        'refund_rate': 100 * safe_div(len(refunds), len(data))
    }

rates_top_1_percent = calculate_rates(data_top_1_percent)
rates_bottom_99_percent = calculate_rates(data_bottom_99_percent)

# Calculate AOV
def calculate_aov(data):
    total_revenue = data['revenue_converted'].sum()
    total_orders = data['transaction_id'].nunique()
    return safe_div(total_revenue, total_orders)

aov_top_1_percent = calculate_aov(data_top_1_percent_clean)
aov_bottom_99_percent = calculate_aov(data_bottom_99_percent_clean)

# Calculate Avg Items per Order
data_ga4_top_1_percent = data_ga4_item[data_ga4_item['user_id'].isin(top_1_percent_user_ids)]
data_ga4_bottom_99_percent = data_ga4_item[data_ga4_item['user_id'].isin(bottom_99_percent_user_ids)]

avg_items_per_order_top_1_percent = safe_div(data_ga4_top_1_percent['item_quantity'].sum() , len(data_ga4_top_1_percent['transaction_id'].unique()))
avg_items_per_order_bottom_99_percent = safe_div(data_ga4_bottom_99_percent['item_quantity'].sum() , len(data_ga4_bottom_99_percent['transaction_id'].unique()))

# Calculate Days Between Orders
def calculate_avg_days_between_orders(data):
    data_sorted = data.sort_values(['user_id', 'datetime'])
    data_sorted['days_between_orders'] = data_sorted.groupby('user_id')['datetime'].diff().dt.days
    return data_sorted['days_between_orders'].mean()

days_between_orders_top_1_percent = calculate_avg_days_between_orders(data_top_1_percent_clean)
days_between_orders_bottom_99_percent = calculate_avg_days_between_orders(data_bottom_99_percent_clean)

def safe_percentage(value):
    """Format a numeric value as a percentage string, handling NaN or inf."""
    if pd.isna(value) or np.isinf(value):
        return ""
    return f"{value:.1f}%"

# Create comparison DataFrame
top_1_percent_metrics_df = pd.DataFrame({
    "Metric": [
        "Number of Clients",
        "Average # of Orders",
        "Min / Max # of Orders",
        "Average LTV",
        "Min / Max LTV",
        "Average First Purchase Value",
        "Min / Max First Purchase Value",
        "Rate of Cancellations (%)",
        "Rate of Refunds (%)",
        "Average Order Value (AOV)",
        "Average Number of Items per Order",
        "Average Number of Days Between Orders"
    ],
    "Top 1% Clients": [
        f"{len(top_1_percent_clients):,.0f}",
        f"{top_1_percent_clients['# of Orders'].mean():.1f}",
        f"{top_1_percent_clients['# of Orders'].min()} / {top_1_percent_clients['# of Orders'].max()}",
        f"${top_1_percent_clients['Grand Total (Base)'].mean():,.1f}",
        f"${top_1_percent_clients['Grand Total (Base)'].min():,.1f} / ${top_1_percent_clients['Grand Total (Base)'].max():,.1f}",
        f"${top_1_percent_clients['First Purchase Value'].mean():,.1f}",
        f"${top_1_percent_clients['First Purchase Value'].min():,.1f} / ${top_1_percent_clients['First Purchase Value'].max():,.1f}",
        safe_percentage(rates_top_1_percent['cancellation_rate']),
        safe_percentage(rates_top_1_percent['refund_rate']),
        f"${aov_top_1_percent:,.1f}",
        f"{avg_items_per_order_top_1_percent:.1f}",
        f"{days_between_orders_top_1_percent:.1f}"
    ],
    "Bottom 99% Clients": [
        f"{len(bottom_99_percent_clients):,.0f}",
        f"{bottom_99_percent_clients['# of Orders'].mean():.1f}",
        f"{bottom_99_percent_clients['# of Orders'].min()} / {bottom_99_percent_clients['# of Orders'].max()}",
        f"${bottom_99_percent_clients['Grand Total (Base)'].mean():,.1f}",
        f"${bottom_99_percent_clients['Grand Total (Base)'].min():,.1f} / ${bottom_99_percent_clients['Grand Total (Base)'].max():,.1f}",
        f"${bottom_99_percent_clients['First Purchase Value'].mean():,.1f}",
        f"${bottom_99_percent_clients['First Purchase Value'].min():,.1f} / ${bottom_99_percent_clients['First Purchase Value'].max():,.1f}",
        safe_percentage(rates_bottom_99_percent['cancellation_rate']),
        safe_percentage(rates_bottom_99_percent['refund_rate']),
        f"${aov_bottom_99_percent:,.1f}",
        f"{avg_items_per_order_bottom_99_percent:.1f}",
        f"{days_between_orders_bottom_99_percent:.1f}"
    ]
})

# Display final comparison
top_1_percent_metrics_df

In [ ]:
# -- 7. CORRELATION ANALYSIS --
# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()

# Ensure numeric data
data_m2_noCancel_noRefund['Grand Total (Base)'] = pd.to_numeric(data_m2_noCancel_noRefund['revenue_converted'], errors='coerce')

# Calculate the first purchase value for each customer
data_m2_noCancel_noRefund['First Purchase Value'] = data_m2_noCancel_noRefund.groupby('user_id')['Grand Total (Base)'].transform('first')

# Group by customer to calculate total and first purchase revenues
client_data = data_m2_noCancel_noRefund.groupby('user_id').agg({
    'Grand Total (Base)': 'sum',  # Lifetime Value (LTV)
    'First Purchase Value': 'first'  # First Purchase Value (FPV)
}).rename(columns={'Grand Total (Base)': 'LTV', 'First Purchase Value': 'FPV'})

# Calculate the correlation
correlation, p_value = pearsonr(client_data['FPV'], client_data['LTV'])

# Find illustrative examples
high_ltv_examples = client_data.sort_values('LTV', ascending=False).head(5)

correlation_results_df = pd.DataFrame({
    'Metric': ['Correlation Coefficient (r)', 'P-value', 'Statistical Significance'],
    'Value': [
        f"{correlation:.4f}",
        f"{p_value:.4e}",
        "Statistically Significant" if p_value < 0.05 else "Not Statistically Significant"
    ],
    'Description': [
        'Pearson correlation between First Purchase Value and Lifetime Value',
        'Probability that the correlation occurred by chance',
        'Significant if p-value < 0.05'
    ]
})

# Output results
correlation_results_df

In [ ]:
# -- 8. FIRST PURCHASE VS. TOTAL REVENUE --

# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()

# Ensure numeric data
data_m2_noCancel_noRefund['Grand Total (Base)'] = pd.to_numeric(data_m2_noCancel_noRefund['revenue_converted'], errors='coerce')

# Calculate the first purchase value for each customer
data_m2_noCancel_noRefund['First Purchase Value'] = data_m2_noCancel_noRefund.groupby('user_id')['Grand Total (Base)'].transform('first')

# Group by customer to calculate total and first purchase revenues
client_data = data_m2_noCancel_noRefund.groupby('user_id').agg({
    'Grand Total (Base)': 'sum',
    'First Purchase Value': 'first'
})

# Calculate metrics
total_revenue = client_data['Grand Total (Base)'].sum()
first_purchase_revenue = client_data['First Purchase Value'].sum()
revenue_less_first_purchase = total_revenue - first_purchase_revenue

# Create a Plotly bar chart
fig_first_purchase_vs_revenue = go.Figure()

# Add bars for first purchase revenue and remaining revenue
fig_first_purchase_vs_revenue.add_trace(go.Bar(
    x=["First Purchase Revenue", "All Revenue less First Purchase"],
    y=[first_purchase_revenue / total_revenue * 100, revenue_less_first_purchase / total_revenue * 100],
    text=[f"{first_purchase_revenue / total_revenue * 100:.1f}%", f"{revenue_less_first_purchase / total_revenue * 100:.1f}%"],  # Rounded to 1 decimal place
    textposition='auto',
    marker=dict(color=['blue', 'lightblue']),
    width=0.35  # Reduce the width of the bars
))

# Customize the layout
fig_first_purchase_vs_revenue.update_layout(
    title="First Purchase vs. Total Revenue",
    xaxis_title="",  # Remove the x-axis title
    yaxis_title="Percentage of Total Revenue",
    yaxis=dict(ticksuffix="%"),
    template="plotly_white",
    width=700,
    height=500,
    bargap=0.3,  # Add some gap between the bars
)

# Show the chart
fig_first_purchase_vs_revenue.show()



In [ ]:
# -- 9. FIRST PURCHASE VS. TOTAL REVENUE --
# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()
# Ensure numeric data
data_m2_noCancel_noRefund['Grand Total (Base)'] = pd.to_numeric(data_m2_noCancel_noRefund['revenue_converted'], errors='coerce')

# Calculate the first purchase value for each customer
data_m2_noCancel_noRefund['First Purchase Value'] = data_m2_noCancel_noRefund.groupby('user_id')['Grand Total (Base)'].transform('first')

# Group by customer to calculate total and first purchase revenues
client_data = data_m2_noCancel_noRefund.groupby('user_id').agg({
    'Grand Total (Base)': 'sum',
    'First Purchase Value': 'first'
})

# Calculate metrics
total_revenue = client_data['Grand Total (Base)'].sum()
first_purchase_revenue = client_data['First Purchase Value'].sum()
revenue_less_first_purchase = total_revenue - first_purchase_revenue

# Calculate percentages
first_purchase_percentage = (first_purchase_revenue / total_revenue) * 100
remaining_revenue_percentage = (revenue_less_first_purchase / total_revenue) * 100

# Create the table as a pandas DataFrame
first_purchase_vs_total_revenue_df = pd.DataFrame({
    "Data": ["First Purchase Revenue", "All Revenue - First Purchase"],
    "% of Revenue": [f"{first_purchase_percentage:.1f}%", f"{remaining_revenue_percentage:.1f}%"],
    "Revenue": [f"${first_purchase_revenue:.0f}", f"${revenue_less_first_purchase:.0f}"]
})


# Display the table
first_purchase_vs_total_revenue_df

In [ ]:
# Group by user origin (traffic_source) and aggregate revenue and quantity
grouped = data_ga4_source.groupby("traffic_source", as_index=False).agg({
    "revenue": "sum",
    "quantity": "sum"
})

# Rename 'traffic_source' to 'User Origin'
grouped.rename(columns={"traffic_source": "User Origin"}, inplace=True)

# Calculate total revenue and total items purchased
total_revenue = grouped["revenue"].sum()
total_items = grouped["quantity"].sum()

# Calculate percentage metrics
grouped["% of Revenue"] = grouped["revenue"] / total_revenue * 100
grouped["% of Items Purchased"] = grouped["quantity"] / total_items * 100

# Optionally, sort by revenue and select the top 10 user origins
grouped.sort_values(by="revenue", ascending=False, inplace=True)
top10 = grouped.head(10)

# Reshape data for plotting
# Convert the dataframe from wide to long format for plotting
top10_long = top10.melt(
    id_vars=["User Origin"],
    value_vars=["% of Revenue", "% of Items Purchased"],
    var_name="Metric",
    value_name="Percentage"
)

# Create the grouped bar chart with specified colors and legend position
# Define a color mapping for the metrics
color_map = {
    "% of Revenue": "lightblue",
    "% of Items Purchased": "lightgreen"
}

fig_user_origin_chart = px.bar(
    top10_long,
    x="User Origin",
    y="Percentage",
    color="Metric",
    barmode="group",  # places bars side-by-side
    title="User Origin Metrics: % of Revenue vs. % of Items Purchased",
    labels={"Percentage": "Percentage (%)"},
    color_discrete_map=color_map  # apply our custom colors
)

# Update layout to place the legend at the top center, remove x-axis title and "Metric" from the legend
fig_user_origin_chart.update_layout(
    xaxis_title='',  # Remove the x-axis title
    legend_title='',  # Remove the legend title ("Metric")
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# Display the figure
fig_user_origin_chart.show()


In [ ]:
#  -- 11. USER ORIGIN TABLE --

# Group by user origin (traffic_source) and aggregate revenue and quantity
grouped = data_ga4_source.groupby("traffic_source", as_index=False).agg({
    "revenue": "sum",
    "quantity": "sum"
})

# Rename 'traffic_source' to 'User Origin'
grouped.rename(columns={"traffic_source": "User Origin"}, inplace=True)

# Calculate totals
total_revenue = grouped["revenue"].sum()
total_items = grouped["quantity"].sum()

# Calculate percentages and average revenue
grouped["% of Revenue"] = grouped["revenue"] / total_revenue * 100
grouped["% of Items Purchased"] = grouped["quantity"] / total_items * 100
grouped["Average Revenue per Item"] = grouped["revenue"] / grouped["quantity"]

# Replace NaN/inf values (e.g., where quantity = 0)
grouped["Average Revenue per Item"] = grouped["Average Revenue per Item"].fillna(0).replace([float('inf'), -float('inf')], 0)

# Format columns
grouped["% of Revenue"] = grouped["% of Revenue"].map("{:.1f}%".format)
grouped["% of Items Purchased"] = grouped["% of Items Purchased"].map("{:.1f}%".format)
grouped["Average Revenue per Item"] = grouped["Average Revenue per Item"].map("${:,.1f}".format)

# Sort by total revenue desc and keep top 10
grouped.sort_values(by="revenue", ascending=False, inplace=True)
top10 = grouped.head(10)

# Create final table (you can include 'revenue' and 'quantity' columns too if needed)
client_origin_df = top10[[
    "User Origin",
    "% of Revenue",
    "% of Items Purchased",
    "Average Revenue per Item"
]]

client_origin_df

In [ ]:
# -- 12. PRODUCTS FOR TOP 10% CUSTOMERS --

"""
Example of comment:
Among the top 10% of customers, 40% began their journey with premium sneakers, 30% with sportswear bundles, and 20% with high-value accessories.
"""
# Remove canceled/refunded, keep only revenue > 0
data_ga4_item_noCancel_noRefund = data_ga4_item[
    ~data_ga4_item['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_ga4_item['revenue'] > 0)
].copy()

# Group by customer to get total revenue
customer_revenue = (
    data_ga4_item_noCancel_noRefund.groupby('user_id', as_index=False)['revenue']
      .sum()
      .rename(columns={'revenue': 'total_revenue'})
)

# Calculate the revenue threshold for top 10%
threshold_10pct = customer_revenue['total_revenue'].quantile(0.90)

# Get the list (subset) of top 10% customers
top_10pct_customers = customer_revenue[
    customer_revenue['total_revenue'] >= threshold_10pct
]['user_id']

# -- 2. Find each top-10%-customer's first purchase category
# We'll only consider rows from those top 10% customers
df_top_10pct = data_ga4_item_noCancel_noRefund[data_ga4_item_noCancel_noRefund['user_id'].isin(top_10pct_customers)].copy()

# Sort by date to identify first purchase
df_top_10pct = df_top_10pct.sort_values(by=['user_id', 'datetime_x'])

# Drop duplicates so each customer is kept at his/her earliest purchase record
df_first_purchase = (
    df_top_10pct
    .drop_duplicates(subset=['user_id'], keep='first')
    .copy()
)

# --3. Calculate the percentage share of each first-purchase category
category_counts = (
    df_first_purchase.groupby(category_to_analyse)
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)

total_first_purchases = category_counts['count'].sum()
category_counts['percentage'] = (category_counts['count'] / total_first_purchases) * 100

# -- 4. Plot horizontal bar chart with Plotly
fig_category_percentage_chart = px.bar(
    category_counts,
    x='percentage',               # numeric axis
    y=category_to_analyse,            # category axis
    orientation='h',             # horizontal bars
    color=category_to_analyse,        # color by category (optional)
    title='Distribution of First Purchase Category (Top 10% Customers)',
    labels={'percentage': '% of First Purchases', category_to_analyse: 'Category'}
)

fig_category_percentage_chart.update_traces(marker_color='lightgreen')  # Change 'blue' to any desired color


# Make layout adjustments for readability
fig_category_percentage_chart.update_layout(
    showlegend=False,  # Hide legend if categories are self-explanatory
    xaxis_tickformat=".0f",  # Format percentages
    yaxis={'categoryorder':'total ascending'},  # Largest bar at top
    bargap=0.3,
    plot_bgcolor="white",  # Set plot background to white
    paper_bgcolor="white"  # Set overall figure background to white
)

fig_category_percentage_chart.show()


In [ ]:
# -- 13. SUMMARY AND CORRELATION CATEGORY ANALYSIS --

"""
Example of comment:
The highest positive correlation is about 0.23 for "Products/FPGA Boards/Expansion Modules/Pmods". That is a mild positive relationship with LTV. It suggests that customers who spend more on that category tend to have higher LTV, but it’s not a very strong correlation.
Many other categories show small (or near-zero) correlations, meaning they don’t appear to strongly drive LTV in a linear sense.
A high transaction count + low correlation likely means that category is universal or low-value. Many customers buy it regardless of whether they go on to become high-LTV or low-LTV.
A smaller transaction count + high positive correlation might indicate a specialized product that predicts or drives higher overall spend.
"""
# Remove canceled/refunded, keep only revenue > 0
data_ga4_item_noCancel_noRefund = data_ga4_item[
    ~data_ga4_item['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), 
                                                case=False, na=False) &
    (data_ga4_item['revenue'] > 0)
].copy()

# Group by customer to get total revenue
customer_revenue = (
    data_ga4_item_noCancel_noRefund.groupby('user_id', as_index=False)['revenue']
      .sum()
      .rename(columns={'revenue': 'total_revenue'})
)

# Calculate the revenue threshold for top 10%
threshold_10pct = customer_revenue['total_revenue'].quantile(0.90)

# Get the list (subset) of top 10% customers
top_10pct_customers = customer_revenue[
    customer_revenue['total_revenue'] >= threshold_10pct
]['user_id']

# -- 2. Find each top-10%-customer's first purchase category
# We'll only consider rows from those top 10% customers
df_top_10pct = data_ga4_item_noCancel_noRefund[
    data_ga4_item_noCancel_noRefund['user_id'].isin(top_10pct_customers)
].copy()

# Identify each user's FIRST purchase (among top 10% only)
df_sorted = df_top_10pct.sort_values(by=['user_id', 'datetime_x'])
df_first_purchase = df_sorted.drop_duplicates(subset=['user_id'], keep='first').copy()

# We'll focus on category_to_analyse
df_first_purchase = df_first_purchase[['user_id', category_to_analyse]].copy()

# Create a one‐hot encoding for first‐purchase categories
df_one_hot = pd.get_dummies(
    df_first_purchase[['user_id', category_to_analyse]],
    columns=[category_to_analyse],
    prefix='firstCat',
    dtype=int
)
df_one_hot.set_index('user_id', inplace=True)

# Get total LTV by user (top 10% only)
df_ltv = (
    df_top_10pct.groupby('user_id', as_index=False)['revenue']
               .sum()
               .rename(columns={'revenue': 'LTV'})
)

# Merge one‐hot table with LTV table
df_merged = df_one_hot.merge(df_ltv, left_index=True, right_on='user_id', how='left')

# Correlation of first‐purchase category with LTV
df_merged_numeric = df_merged.drop(columns=['user_id']).select_dtypes(['int', 'float'])
corr_matrix = df_merged_numeric.corr()
corr_with_ltv = corr_matrix['LTV'].drop('LTV')  # exclude LTV row itself

# Count how many users had each first‐purchase category (in top 10%)
df_cat_counts = (
    df_first_purchase.groupby(category_to_analyse)
                     .size()
                     .reset_index(name='num_users')
)
total_first_purchases = df_cat_counts['num_users'].sum()
df_cat_counts['pct_total'] = (df_cat_counts['num_users'] / total_first_purchases) * 100

# Convert correlation result into DataFrame & map dummy columns back
corr_df = corr_with_ltv.to_frame(name='correlation_with_ltv').reset_index()
corr_df[category_to_analyse] = corr_df['index'].str.replace('^firstCat_', '', regex=True)

category_repartition_df = pd.merge(
    df_cat_counts,
    corr_df,
    on=category_to_analyse,
    how='left'
)

# Rename columns to highlight Top 10% context
category_repartition_df.rename(columns={
    category_to_analyse: 'Category Name',
    'num_users': 'Num First Purchases (Top 10%)',
    'pct_total': '% of First Purchases (Top 10%)',
    'correlation_with_ltv': 'Correlation with LTV (Top 10%)'
}, inplace=True)

# Sort and tidy
category_repartition_df.sort_values(by='Num First Purchases (Top 10%)', ascending=False, inplace=True)
category_repartition_df['% of First Purchases (Top 10%)'] = (category_repartition_df['% of First Purchases (Top 10%)'].round(1)).astype(str) + '%'
category_repartition_df['Correlation with LTV (Top 10%)'] = category_repartition_df['Correlation with LTV (Top 10%)'].round(2)

# Final summary
category_repartition_df = category_repartition_df[['Category Name',
            '% of First Purchases (Top 10%)', 'Correlation with LTV (Top 10%)']].head(10)

category_repartition_df

In [ ]:
# -- 14. Ratio discount over time --
# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()


# Here we designate an order as 'Discounted' if it has a discount value > 0 or a discount code exists.
data_m2_noCancel_noRefund['order_type'] = data_m2_noCancel_noRefund.apply(
    lambda row: 'Discounted' if (row['discount'] > 0) else 'Full Price',
    axis=1
)

# Create a new column 'order_month' by extracting the month from 'datetime'
# The .to_period('M') converts to a monthly period, and .to_timestamp() converts it back to a datetime for plotting.
data_m2_noCancel_noRefund['order_month'] = data_m2_noCancel_noRefund['datetime'].dt.to_period('M').dt.to_timestamp()

# Group the data by order_month and order_type to get monthly order counts
monthly_trend = data_m2_noCancel_noRefund.groupby(['order_month', 'order_type']).agg(
    order_count=('transaction_id', 'count')
).reset_index()

# Pivot the table to have separate columns for each order type’s count per month
monthly_count_pivot = monthly_trend.pivot(index='order_month', columns='order_type', values='order_count').fillna(0)

# --- Calculate the Discount Ratio ---
# Ensure that both columns exist; if one of the types is missing for a month, fill with 0.
# Ratio = (Discounted orders) / (Discounted orders + Full Price orders)
monthly_count_pivot['discount_ratio'] = (
    monthly_count_pivot.get('Discounted', 0) / 
    (monthly_count_pivot.get('Discounted', 0) + monthly_count_pivot.get('Full Price', 0))
)

# --- Generate a Line Chart for the Monthly Discount Ratio ---
fig_ratio_discount = go.Figure()

fig_ratio_discount.add_trace(go.Scatter(
    x=monthly_count_pivot.index,
    y=monthly_count_pivot['discount_ratio'],
    mode='lines',
    name='Discount Ratio',
    line=dict(color='orange')
))

# Customize the layout
fig_ratio_discount.update_layout(
    title='Monthly Trend of Ratio: Discounted Transactions / Total Transactions',
    yaxis_title='Discount Ratio',
    yaxis=dict(tickformat='.0%'), 
    template='plotly_white',
    width=900,
    height=500
)

fig_ratio_discount.show()


In [ ]:
# -- 15. Customer Segmentation Discount --
# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()


# --- Prepare the data ---
# Create a new column 'order_type' that designates whether an order was discounted.
data_m2_noCancel_noRefund['order_type'] = data_m2_noCancel_noRefund.apply(
    lambda row: 'Discounted' if (row['discount'] > 0) else 'Full Price', 
    axis=1
)

# Group the data by order_type to calculate:
# - The count of orders
# - Total revenue
order_summary = data_m2_noCancel_noRefund.groupby('order_type').agg(
    order_count=('transaction_id', 'count'),
    total_revenue=('revenue_converted', 'sum')
).reset_index()

# Calculate total revenue and total transactions
total_revenue = order_summary['total_revenue'].sum()
total_orders = order_summary['order_count'].sum()

# Calculate percentage metrics
order_summary['% of Revenue'] = (order_summary['total_revenue'] / total_revenue) * 100
order_summary['% of Transactions'] = (order_summary['order_count'] / total_orders) * 100

# Reshape the data for plotting
# Convert the dataframe from wide to long format for plotting
order_summary_long = order_summary.melt(
    id_vars=["order_type"],
    value_vars=["% of Revenue", "% of Transactions"],
    var_name="Metric",
    value_name="Percentage"
)

# --- Generate the grouped bar chart ---
fig_share_discount = go.Figure()

# Define a color mapping for the metrics
color_map = {
    "% of Revenue": "lightblue",
    "% of Transactions": "lightgreen"
}

# Add bars for % of Revenue (left side)
fig_share_discount.add_trace(go.Bar(
    x=order_summary_long['order_type'],
    y=order_summary_long[order_summary_long['Metric'] == "% of Revenue"]['Percentage'],
    name="% of Revenue",  # Change legend name
    marker=dict(color="lightblue"),
    offsetgroup=0,  # Ensure this group is on the left side
    width=0.35,  # Reduce the bar width
    text=order_summary_long[order_summary_long['Metric'] == "% of Revenue"]['Percentage'].round(1).astype(str) + '%',  # Add the label text
    textposition='outside',  # Position the text outside the bar (top)
))

# Add bars for % of Transactions (right side)
fig_share_discount.add_trace(go.Bar(
    x=order_summary_long['order_type'],
    y=order_summary_long[order_summary_long['Metric'] == "% of Transactions"]['Percentage'],
    name="% of Transactions",  # Change legend name
    marker=dict(color="lightgreen"),
    offsetgroup=1,  # Ensure this group is on the right side
    width=0.35,  # Reduce the bar width
    text=order_summary_long[order_summary_long['Metric'] == "% of Transactions"]['Percentage'].round(1).astype(str) + '%',  # Add the label text
    textposition='outside',  # Position the text outside the bar (top)
))

# Customize the layout
fig_share_discount.update_layout(
    title="Discount vs Full Price Transactions",
    yaxis_title="Percentage (%)",
    barmode="group",  # places bars side-by-side
    template="plotly_white",
    width=900,
    height=500,
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    bargap=0.2,  # Add small gap between bars (adjust value for more or less space)
    bargroupgap=0.3  # Adjust gap between groups
)

fig_share_discount.show()


In [ ]:
# -- 16.Customer Segmentation Discount --
# Remove canceled/refunded, keep only revenue > 0
data_m2_noCancel_noRefund = data_m2[
    ~data_m2['order_status'].str.contains("|".join(order_status_keyWords_Refund_Cancel), case=False, na=False) & 
    (data_m2['revenue_converted'] > 0)
].copy()

# Create a new column 'order_type' that designates whether an order was discounted.
data_m2_noCancel_noRefund['order_type'] = data_m2_noCancel_noRefund.apply(
    lambda row: 'Discounted' if (row['discount'] > 0 ) else 'Full Price', 
    axis=1
)

# Group orders by customer (using 'user_name' or another unique identifier)
customer_discount = data_m2_noCancel_noRefund.groupby('user_id').agg(
    total_orders=('transaction_id', 'count'),
    discounted_orders=('order_type', lambda x: (x == 'Discounted').sum()),
    fullprice_orders=('order_type', lambda x: (x == 'Full Price').sum()),
    total_revenue=('revenue_converted', 'sum')
).reset_index()

# Classify each customer
customer_discount['customer_segment'] = customer_discount.apply(
    lambda row: 'Discount Only' if row['discounted_orders'] == row['total_orders'] 
                else ('Full Price Only' if row['fullprice_orders'] == row['total_orders'] else 'Mixed'),
    axis=1
)

# Create a summary table by segment
customer_segment_discount_df = customer_discount.groupby('customer_segment').agg(
    number_of_customers=('user_id', 'count'),
    total_revenue=('total_revenue', 'sum'),
    average_revenue_per_customer=('total_revenue', 'mean')
).reset_index()

# Calculate percentage values manually
customer_segment_discount_df['percentage_customers'] = (customer_segment_discount_df['number_of_customers'] / customer_segment_discount_df['number_of_customers'].sum()) * 100

# --- Generate a pie chart to visualize customer segmentation ---
fig_client_distribution_discount = go.Figure(data=[go.Pie(
    labels=customer_segment_discount_df['customer_segment'],
    values=customer_segment_discount_df['percentage_customers'],
    hole=0.5,  # Increase the hole size to 50%
    marker=dict(colors=['#66c2a5', '#fc8d62', '#8da0cb']),  # Custom colors
    textinfo='percent',  # Display the percentage
    texttemplate='%{value:.1f}%',  # Manually format percentages with 1 decimal place
)])

fig_client_distribution_discount.update_layout(
    title='Discount vs Full Price Buyers',
    template='plotly_white'
)

# Render the pie chart
fig_client_distribution_discount.show()


In [ ]:
# -- Customer Segmentation Discount Table --

# Calculate percentage values manually
customer_segment_discount_df['percentage_customers'] = (customer_segment_discount_df['number_of_customers'] / customer_segment_discount_df['number_of_customers'].sum()) * 100

# Format values to 1 decimal place for all columns
customer_segment_discount_df['percentage_customers'] = customer_segment_discount_df['percentage_customers'].round(1).astype(str) + '%'  # Add '%' sign
customer_segment_discount_df['total_revenue'] = '$' + customer_segment_discount_df['total_revenue'].round(1).astype(str)  # Add '$' sign and convert to string
customer_segment_discount_df['average_revenue_per_customer'] = '$' + customer_segment_discount_df['average_revenue_per_customer'].round(1).astype(str)  # Add '$' sign and convert to string

# Rename columns to something more natural
customer_segment_discount_df.rename(columns={
    'customer_segment': 'Client Segment',
    'number_of_customers': 'Number of Clients',
    'percentage_customers': '% of Clients',
    'total_revenue': 'Total Revenue',
    'average_revenue_per_customer': 'Avg Revenue per Client'
}, inplace=True)

# Display the final summary table
customer_segment_discount_df


In [ ]:
# -- Save the output in folder [datetime]_results --

# Get the current date and time formatted as YYYYMMDD_HHMMSS
current_datetime = datetime.now().strftime('%Y%m%d_%H%M')
# Define the folder path as [datetime]_revenue
output_dir = f'{current_datetime}_results/'
# Create the results directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
    
    
# -- Export all tables to a single Excel file --
# Use ExcelWriter to create a file with multiple sheets
output_excel_path = os.path.join(output_dir, "combined_results.xlsx")
with pd.ExcelWriter(output_excel_path) as writer:
    # Write each DataFrame to a separate sheet in the Excel file
    client_repartition_df.to_excel(writer, sheet_name='client_repartition_df', index=False)
    top_1_percent_metrics_df.to_excel(writer, sheet_name='top_1_percent_metrics_df', index=False)
    correlation_results_df.to_excel(writer, sheet_name='correlation_results_df', index=False)
    first_purchase_vs_total_revenue_df.to_excel(writer, sheet_name='first_purchase_vs_total_revenue_df', index=False)
    client_origin_df.to_excel(writer, sheet_name='client_origin_df', index=False)
    customer_segment_discount_df.to_excel(writer, sheet_name='customer_segment_discount_df', index=False)
    category_repartition_df.to_excel(writer, sheet_name='category_repartition_df', index=False)


# Output confirmation
print(f"All tables have been successfully written to {output_excel_path}")


# -- Export all figures to PNG files --
figures = {
    "1-revenue_timeline": fig_revenue_timeline,
    "2-revenue_month_and_weekday": fig_revenue_distributin_month_and_weekday,
    "3-client_revenue": fig_client_revenue,  
    "4-top_client_revenue": fig_top_client_revenue,  
    "5-top_client_transactions": fig_top_client_transactions,  
    "6-first_purchase_vs_revenue": fig_first_purchase_vs_revenue, 
    "7-user_origin_chart": fig_user_origin_chart,
    "8-ratio_discount": fig_ratio_discount, 
    "9-share_discountt": fig_share_discount, 
    "10-client_distribution_discount": fig_client_distribution_discount, 
    "11-category_percentage_chart": fig_category_percentage_chart, 
}  

# Save each figure as a PNG in the /results/ folder
for fig_name, fig_obj in figures.items():
    fig_file_path = os.path.join(output_dir, f"{fig_name}.png")
    pio.write_image(fig_obj, fig_file_path)

# Output confirmation
print("All figures have been successfully saved as PNG files in the 'results' folder.")

